<a href="https://colab.research.google.com/github/estivenalv/proyectoIA/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importe de datos

In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 60% 12.0M/20.1M [00:00<00:00, 54.3MB/s]
100% 20.1M/20.1M [00:00<00:00, 76.2MB/s]


In [4]:
!unzip udea*.zip > /dev/null

In [5]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


# Importe de librerias para trabajar el dataframe

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv("train.csv")
print(df.shape)
df.head()

(692500, 12)


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [8]:
dt = pd.read_csv("test.csv")
print(dt.shape)
dt.head()

(296786, 12)


,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


# Limpieza del data frame

In [9]:
# organizo y confirmo que todos mis datos categoricos sean de tipo str para poder trabajar la columna 'ESTU_PRGM_ACADEMICO'
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].astype(str)

# hacer un diccionario con la columna 'ESTU_PRGM_ACADEMICO' donde se pueda generar unas categorias y disminuir la cantidad de valores unicos
ESTU_PRGM_ACADEMICO = {'INGENIERIAS':['INGENIERIA INDUSTRIAL','INGENIERIA CIVIL','INGENIERIA DE SISTEMAS','INGENIERIA AMBIENTAL','INGENIERIA MECANICA','INGENIERIA ELECTRONICA','INGENIERÍA INDUSTRIAL','INGENIERIA QUIMICA',
                                     'INGENIERÍA AMBIENTAL','INGENIERIA ELECTRICA','INGENIERIA MECATRONICA','INGENIERIA AGRONOMICA', 'INGENIERIA BIOMEDICA','INGENIERIA DE ALIMENTOS','INGENIERÍA CIVIL','INGENIERIA AMBIENTAL Y SANITARIA',
                                     'INGENIERIA DE TELECOMUNICACIONES','INGENIERIA DE SISTEMAS Y COMPUTACION','INGENIERIA AGROINDUSTRIAL','INGENIERIA ELECTROMECANICA','INGENIERIA INFORMATICA','INGENIERIA DE PETROLEOS','INGENIERIA DE PRODUCCION',
                                     'INGENIERIA FINANCIERA','INGENIERÍA DE SISTEMAS','INGENIERÍA DE SOFTWARE','INGENIERIA EN SEGURIDAD Y SALUD PARA EL TRABAJO',
                                     'INGENIERIA DE SOFTWARE', 'INGENIERIA AGRICOLA','INGENIERIA FORESTAL','INGENIERIA EN TELECOMUNICACIONES','INGENIERIA ADMINISTRATIVA',
                                     'INGENIERIA EN SEGURIDAD INDUSTRIAL E HIGIENE OCUPACIONAL','INGENIERIA AERONAUTICA','INGENIERIA DE MINAS','INGENIERÍA MECATRÓNICA',
                                     'INGENIERIA DE PRODUCTIVIDAD Y CALIDAD','INGENIERÍA ELECTROMECÁNICA','INGENIERIA ELECTRONICA Y TELECOMUNICACIONES', 'INGENIERÍA ELECTRÓNICA', 'INGENIERIA FISICA', 'INGENIERIA AGROPECUARIA', 'INGENIERIA  CIVIL','INGENIERIA METALURGICA',
                                     'INGENIERIA FINANCIERA Y DE NEGOCIOS','INGENIERIA GEOLOGICA','INGENIERIA EN MECATRONICA','INGENIERIA CATASTRAL Y GEODESIA',
                                     'INGENIERÍA MECÁNICA','INGENIERIA AGROECOLOGICA', 'INGENIERÍA INFORMÁTICA','INGENIERIA TOPOGRAFICA','INGENIERÍA ELÉCTRICA', 'INGENIERIA DE PROCESOS', 'INGENIERIA EN HIGIENE Y SEGURIDAD OCUPACIONAL', 'INGENIERIA DE SISTEMAS Y TELECOMUNICACIONES', 'INGENIERIA DE DISEÑO DE PRODUCTO',
                                     'INGENIERIA AMBIENTAL Y DE SANEAMIENTO','INGENIERÍA AGROINDUSTRIAL', 'INGENIERIA AGROFORESTAL','INGENIERIA MULTIMEDIA', 'INGENIERIA DE SISTEMAS E INFORMATICA',
                                     'BIOINGENIERIA','INGENIERIA DE SONIDO','INGENIERIA SANITARIA Y AMBIENTAL','INGENIERIA EN TELEMATICA','INGENIERIA EN MULTIMEDIA',
                                     'INGENIERIA COMERCIAL','INGENIERÍA AGRONÓMICA','INGENIERÍA MECATRÓNICA - CICLO PROFESIONAL UNIVERSITARIO','INGENIERÍA DE SISTEMAS Y COMPUTACIÓN', 'INGENIERÍA QUÍMICA', 'INGENIERÍA DE TELECOMUNICACIONES', 'INGENIERÍA SANITARIA', 'INGENIERIA DE TRANSPORTE Y VIAS', 'CONSTRUCCION EN ARQUITECTURA E INGENIERIA',
                                     'INGENIERIA DE MATERIALES','INGENIERÍA BIOQUÍMICA','INGENIERÍA DE PRODUCCIÓN','BIOINGENIERÍA','INGENIERIA BIOTECNOLOGICA','INGENIERIA SANITARIA',
                                     'INGENIERÍA EN DISEÑO DE ENTRETENIMIENTO DIGITAL','INGENIERÍA GEOLÓGICA','INGENIERIA EN HIGIENE Y SEGURIDAD INDUSTRIAL','INGENIERIA  AMBIENTAL',
                                     'INGENIERIA GEOGRAFICA Y AMBIENTAL','INGENIERÍA EN DISEÑO INDUSTRIAL','INGENIERIA DE PROCESOS INDUSTRIALES','INGENIERIA AGRONÓMICA',
                                     'INGENIERIA DE MINAS Y METALURGIA','INGENIERIA EN  TELECOMUNICACIONES','INGENIERIA EN CONTROL','INGENIERIA DE MERCADOS','INGENIERIA BIOLOGICA',
                                     'INGENIERIA EN ENERGIA','INGENIERÍA GEOGRÁFICA Y AMBIENTAL','INGENIERIA DE SISTEMAS Y  COMPUTACION','INGENIERIA EN AUTOMATIZACIÓN','INGENIERIA PESQUERA',
                                     'INGENIERIA EN AUTOMATICA INDUSTRIAL','INGENIERÌA INFORMÀTICA','INGENIERÍA AGROPECUARIA','INGENIERÍA ELECTRÓNICA Y TELECOMUNICACIONES','INGENIERIA EN INSTRUMENTACION Y CONTROL',
                                     'INGENIERÍA DE PETRÓLEO Y GAS','INGENIERIA EN INFORMATICA','INGENIERIA DE CONTROL','INGENIERÍA MULTIMEDIA','INGENIERÍA AMBIENTAL Y SANITARIA',
                                     'INGENIERÍA FORESTAL','INGENIERIA EN PRODUCCION ACUICOLA','INGENIERIA DE TELECOMUNICACIONES E INFORMATICA','INGENIERIA EN SOFTWARE','INGENIERÍA AERONÁUTICA',
                                     'INGENIERÍA DE LAS ARTES DIGITALES','INGENIERÍA DE ALIMENTOS','INGENIERÍA EN PROCESOS INDUSTRIALES','INGENIERIA EN NANOTECNOLOGIA','INGENIERÍA EN ENERGÍAS',
                                     'INGENIERIA  AGROFORESTAL','INGENIERIA DE SISTEMAS DE INFORMACION','INGENIERIA NAVAL','INGENIERÍA EN TECNOLOGÍAS DE LA INFORMACIÓN Y LAS COMUNICACIONES',
                                     'INGENIERÍA EN ENERGÍA','INGENIERIA TELEMATICA','INGENIERIA MATEMATICA','INGENIERIA PECUARIA','INGENIERIA DE PRODUCCION AGROINDUSTRIAL','INGENIERÍA EN INFORMÁTICA',
                                     'INGENIERIA TELEMÁTICA','INGENIERIA EN TOPOGRAFIA','INGENIERIA BIOMÉDICA','INGENIERÍA EN AUTOMATIZACIÓN','INGENIERÍA EN TELEMÁTICA','INGENIERÍA DE PETRÓLEOS',
                                     'INGENIERIA DE MANTENIMIENTO','INGENIERÍA EN PROCESOS AGROINDUSTRIALES','INGENIERIA MECATRONICO','INGENIERÍA AGRONOMICA','INGENIERÍA NAVAL','INGENIERIA','INGENIERIA',
                                     'INGENIERÍA OCEANOGRÁFICA','INGENIERIA DE SISTEMAS INFORMÁTICOS','INGENIERÍA EN SEGURIDAD Y SALUD PARA EL TRABAJO','INGENIERIA MECATRÓNICA','INGENIERIA DE SOFTWARE Y COMUNICACIONES',
                                     'INGENIERIA DE PLASTICOS','INGENIERIA EN ENERGIAS','INGENIERIA ELÉCTRICA','INGENIERÍA URBANA','INGENIERÍA PETROQUÍMICA','INGENIERIA EN CONTROL Y AUTOMATIZACIÓN',
                                     'INGENIERIA TEXTIL','INGENIERÍA AGROECOLÓGICA','INGENIERÍA EN LOGISTICA Y OPERACIONES','INGENIERIA MECÁNICA','INGENIERÍA AGRÍCOLA','INGENIERÍA BIOMÉDICA',
                                     'INGENIERIA DE SISTEMAS CON ENFASIS EN TELECOMUNICACIONES','INGENIERIA EN AUTOMATIZACION INDUSTRIAL','INGENIERIA EN DISEÑO INDUSTRIAL','INGENIERÍA EN SEGURIDAD Y SALUD EN EL TRABAJO',
                                     'INGENIERIA EN DISEÑO DE MAQUINAS Y PRODUCTOS INDUSTRIALES','INGENIERÍA DE MATERIALES','INGENIERÍA DE MINAS','BIOTECNOLOGIA','QUÍMICA INDUSTRIAL','QUIMICA INDUSTRIAL','INGENIER¿A EN SOFTWARE',
                                     'INGENIER¿A EL¿CTRICA','INGENIER¿A INDUSTRIAL','INGENIER¿A DE SISTEMAS'],
                       'LICENCIATURAS':['LICENCIATURA EN PEDAGOGIA INFANTIL','LICENCIATURA EN PEDAGOGÍA INFANTIL','LICENCIATURA EN CIENCIAS SOCIALES','LICENCIATURA EN EDUCACION PREESCOLAR',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTES','LICENCIATURA EN EDUCACION INFANTIL','LICENCIATURA EN ETNOEDUCACION',
                                        'LICENCIATURA EN MATEMATICAS','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICA HUMANIDADES Y LENGUA CASTELLANA','LICENCIATURA EN EDUCACIÓN FÍSICA, RECREACIÓN Y DEPORTE',
                                        'LICENCIATURA EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL','LICENCIATURA EN INGLES','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS SOCIALES',
                                        'LICENCIATURA EN MUSICA','LICENCIATURA EN EDUCACION FISICA','LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES','LICENCIATURA EN LENGUAS MODERNAS','LICENCIATURA EN ESPAÑOL Y LITERATURA'
                                        'LICENCIATURA EN EDUCACIÓN INFANTIL','LICENCIATURA  PEDAGOGIA INFANTIL','LICENCIATURA EN EDUCACIÓN PARA LA PRIMERA INFANCIA','LICENCIATURA EN EDUCACION FISICA Y DEPORTES',
                                        'LICENCIATURA EN LENGUA CASTELLANA E INGLES','LICENCIATURA EN BIOLOGIA Y QUIMICA','LICENCIATURA EN INGLÉS COMO LENGUA EXTRANJERA',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL','LICENCIATURA EN EDUCACION ARTISTICA','LICENCIATURA EN LENGUAS EXTRANJERAS',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTES','LICENCIATURA EN FILOSOFIA','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES- LENGUA CASTELLANA',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA','LICENCIATURA EN EDUCACIÓN BÁSICA PRIMARIA','LICENCIATURA EN EDUCACION ESPECIAL','LICENCIATURA EN PREESCOLAR',
                                        'LICENCIATURA EN LENGUA CASTELLANA','LICENCIATURA EN CULTURA, FISICA, RECREACION Y DEPORTES','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES,LENGUA CASTELLANA E INGLES',
                                        'LICENCIATURA EN MATEMATICAS Y FISICA','LICENCIATURA EN ETNOEDUCACION CON ENFASIS EN CIENCIAS SOCIALES','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICAS','LICENCIATURA EN BIOLOGIA',
                                        'LICENCIATURA EN LENGUA CASTELLANA,INGLES Y FRANCES','LICENCIATURA EN LENGUA CASTELLANA Y LITERATURA','LICENCIATURA EN LENGUA CASTELLANA Y COMUNICACION','LICENCIATURA EN ARTES VISUALES',
                                        'LICENCIATURA EN LENGUAS EXTRANJERAS INGLES - FRANCES','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN EDUCACIÓN ARTÍSTICA','LICENCIATURA EN IDIOMAS EXTRANJEROS',
                                        'LICENCIATURA EN EDUCACION BASICA EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL','LICENCIATURA EN EDUCACION PARA LA PRIMERA INFANCIA','LICENCIATURA EN LITERATURA Y LENGUA CASTELLANA',
                                        'LICENCIATURA EN QUIMICA','LICENCIATURA EN EDUCACION BASICA ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTE','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN INFORMATICA',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN LENGUA CASTELLANA','LICENCIATURA EN FISICA','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES-INGLES','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN INGLÉS',
                                        'LICENCIATURA EN LINGUISTICA Y LITERATURA','LICENCIATURA EN MATEMÁTICAS','LICENCIATURA EN TEOLOGIA','LICENCIATURA EN INFORMATICA','LICENCIATURA EN BILINGÜISMO CON ÉNFASIS EN INGLÉS',
                                        'LICENCIATURA EN COMUNICACION E INFORMATICA EDUCATIVA','LICENCIATURA EN FILOSOFIA Y EDUCACION RELIGIOSA','LICENCIATURA EN LENGUAS EXTRANJERAS: INGLES-FRANCES',
                                        'LICENCIATURA EN INFORMATICA Y MEDIOS AUDIOVISUALES','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION ARTISTICA','LICENCIATURA EN EDUCACION FISICA,RECREACION Y DEPORTES',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN INGLES','LICENCIATURA EN PSICOLOGIA Y PEDAGOGIA','LICENCIATURA EN EDUCACION FISICA,RECREACION Y DEPORTE','LICENCIATURA EN LITERATURA',
                                        'LICENCIATURA EN EDUCACIÓN ESPECIAL','LICENCIATURA EN EDUCACION BÁSICA CON ÉNFASIS EN CIENCIAS NATURALES Y EDUCACIÓN AMBIENTAL','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES -  LENGUA CASTELLANA',
                                        'LICENCIATURA EN FILOSOFIA Y LETRAS','LICENCIATURA EN EDUCACION CON ENFASIS EN EDUCACION ESPECIAL','LICENCIATURA EN INGLES Y FRANCES','LICENCIATURA EN LENGUAS EXTRANJERAS INGLES-FRANCES',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN TECNOLOGIA E INFORMATICA','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES E IDIOMAS','LICENCIATURA EN EDUCACION CON ENFASIS EN INGLES',
                                        'LICENCIATURA EN DEPORTE','LICENCIATURA EN IDIOMAS MODERNOS ESPAÑOL- INGLES','LICENCIATURA EN PSICOPEDAGOGIA CON ENFASIS EN ASESORIA EDUCATIVA','LICENCIATURA EN ARTES','LICENCIATURA EN PEDAGOGIA DE LA PRIMERA INFANCIA',
                                        'LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTE','LICENCIATURA EN INGLÉS','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, ESPAÑOL Y LENGUAS EXTRANJERAS',
                                        'LICENCIATURA EN INGLÉS-ESPAÑOL','LICENCIATURA EN EDUCACION ARTISTICA Y CULTURAL','LICENCIATURA EN HUMANIDADES Y LENGUA CASTELLANA','LICENCIATURA EN EDUCACION COMUNITARIA CON ENFASIS EN DERECHOS HUMANOS',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: ESPAÑOL E INGLES','LICENCIATURA EN EDUCACIÓN ARTÍSTICA','LICENCIATURA EN INGLÉS Y BILINGÜISMO','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, LENGUA CASTELLANA',
                                        'LICENCIATURA EN ARTES ESCENICAS','LICENCIATURA EN PEDAGOGÍA SOCIAL PARA LA REHABILITACIÓN','LICENCIATURA EN CIENCIAS SOCIALES CON ÉNFASIS EN EDUCACIÓN BÁSICA','LICENCIATURA EN EDUCACIÓN FÍSICA Y DEPORTES',
                                        'LICENCIATURA EN LENGUA EXTRANJERA INGLES','LICENCIATURA EN EDUCACION FISICA, DEPORTES Y RECREACION','LICENCIATURA EN EDUCACION  FISICA RECREACION  Y  DEPORTE','LICENCIATURA EN INGLES - ESPAÑOL',
                                        'LICENCIATURA EN EDUCACION BILINGUE CON ENFASIS EN LA ENSEÑANZA DEL INGLES','LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTE','LICENCIATURA EN CIENCIAS RELIGIOSAS','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN LENGUAS EXTRANJERAS',
                                        'LICENCIATURA EN EDUCACI¿N B¿SICA PRIMARIA','LICENCIATURA EN CIENCIAS DEL DEPORTE','LICENCIATURA EN ETNOEDUCACION Y DESARROLLO COMUNITARIO','LICENCIATURA EN EDUCACION: ARTES PLASTICAS','LICENCIATURA EN LENGUAS MODERNAS, INGLES Y FRANCES',
                                        'LICENCIATURA EN IDIOMA EXTRANJERO - INGLÉS','LICENCIATURA EN INFORMATICA EDUCATIVA','LICENCIATURA EN ESPAÑOL E INGLES','LICENCIATURA EN HISTORIA','LICENCIATURA EN EDUCACION ARTISTICA Y ARTES ESCENICAS',
                                        'LICENCIATURA EN CIENCIAS NATURALES Y EDUCACIÓN AMBIENTAL','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: LENGUA CASTELLANA E INGLES','LICENCIATURA EN ESPAÑOL Y LENGUAS EXTRANJERAS','Licenciatura en Lenguas Extranjeras con Énfasis en Inglés',
                                        'LICENCIATURA EN EDUCACION INDUSTRIAL','LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL','LICENCIATURA EN LENGUAS MODERNAS INGLES-FRANCES','LICENCIATURA EN ARTES PLASTICAS',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION ARTISTICA-MUSICA','LICENCIATURA EN EDUCACI¿N ART¿STICA','LICENCIATURA EN EDUCACIÓN  FÍSICA, RECREACIÓN Y DEPORTES','LICENCIATURA EN EDUCACION RELIGIOSA','LICENCIATURA EN ARTE Y FOLKLORE Y CULTURA',
                                        'LICENCIATURA EN PEDAGOGIA  INFANTIL','LICENCIATURA EN TEATRO','LICENCIATURA EN EDUCACIÓN PARA LA CONVIVENCIA Y EL DESARROLLO HUMANO','LICENCIATURA EN PEDAGOG¿A INFANTIL',
                                        'LICENCIATURA EN DISEÑO TECNOLOGICO CON ENFASIS EN SISTEMAS MECANICOS','LICENCIATURA EN RECREACION','LICENCIATURA EN LENGUAS EXTRANJERAS CON ÉNFASIS EN INGLÉS','LICENCIATURA EN EDUCACION AMBIENTAL','LICENCIATURA EN TEOLOGÍA',
                                        'LICENCIATURA EN EDUCACION BASICA ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL','LICENCIATURA EN LENGUAS  EXTRANJERAS','LICENCIATURA EN EDUCACI¿N CON ¿NFASIS EN CIENCIAS SOCIALES Y AMBIENTALES','LICENCIATURA EN CIENCIAS NATURALES',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN LENGUA CASTELLANA E INGLES','LICENCIATURA EN LENGUAS MODERNAS CON ÉNFASIS EN INGLÉS','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTE',
                                        'LICENCIATURA EN LENGUA INGLESA','LICENCIATURA EN CIENCIAS SOCIALES Y DESARROLLO LOCAL','LICENCIATURA EN PRODUCCION AGROPECUARIA','LICENCIATURA EN LENGUAS MODERNAS CON ENFASIS EN INGLES Y FRANCES',
                                        'LICENCIATURA EN ARTES ESCENICAS CON ENFASIS EN TEATRO','LICENCIATURA EN CIENCIAS DEL DEPORTE Y LA EDUCACION FISICA','LICENCIATURA EN BILINGÜISMO CON ENFASIS EN LA ENSEÑANZA DEL INGLÉS','LICENCIATURA EN BIOLOGIA CON ENFASIS EN EDUCACION AMBIENTAL',
                                        'LICENCIATURA EN FILOSOFIA, PENSAMIENTO POLITICO Y ECONOMICO','LICENCIATURA EN ARTES ESCÉNICAS','LICENCIATURA EN EDUCACION BASICA EN DANZA','LICENCIATURA EN EDUCACIÓN FÍSICA Y DEPORTE','LICENCIATURA EN FILOSOFÍA','LICENCIATURA EN FILOSOFIA E HISTORIA',
                                        'LICENCIATURA EN ARTE DRAMATICO','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN CIENCIAS SOCIALES','LICENCIATURA EN FILOSOFIA Y LENGUA CASTELLANA','LICENCIATURA EN EDUCACIÓN  FÍSICA, RECREACIÓN Y DEPORTE','LICENCIATURA EN EDUCACION ARTISTICA CON ENFASIS EN DANZA Y TEATRO',
                                        'LICENCIATURA  EN MATEMATICAS','LICENCIATURA  EN INGLES Y FRANCES','LICENCIATURA EN EDUCACION BILINGUE-ESPAÑOL-INGLES','LICENCIATURA EN EDUCACI¿N INFANTIL','LICENCIATURA EN EDUCACIÓN FÍSICA DEPORTES Y RECREACIÓN','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES LENGUA CASTELLANA',
                                        'LICENCIATURA EN  FILOSOFIA Y CIENCIAS RELIGIOSAS','LICENCIATURA EN LENGUAS EXTRANJERAS CON ¿NFASIS EN INGL¿S','LICENCIATURA EN MATEMATICAS Y ESTADISTICA','LICENCIATURA EN TECNOLOGIA E INFORMATICA','LICENCIATURA EN EDUCACIÓN PREESCOLAR','LICENCIATURA EN  CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
                                        'LICENCIATURA EN ELECTRONICA','LICENCIATURA M¿SICA','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUAS','LICENCIATURA EN EDUCACION BASICA  CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA','LICENCIATURA EN EDUCACIÓN BÁSICA, CON ÉNFASIS EN HUMANIDADES, LENGUA CASTELLANA',
                                        'LICENCIATURA EN LENGUAS EXTRANJERAS CON ENFASIS EN INGLES','LICENCIATURA EN MATEMATICAS E INFORMATICA','LICENCIATURA EN EDUCACIÓN BILINGUE','LICENCIATURA EN EDUCACION POPULAR',
                                        'LICENCIATURA EN PEDAGOGÍA DE LA MADRE TIERRA','LICENCIATURA EN EDUCACION BASICA PRIMARIA','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN CIENCIAS NATURALES Y EDUCACIÓN AMBIENTAL','LICENCIATURA EN  MATEMATICAS Y FISICA','LICENCIATURA EN EDUCACIÓN ARTÍSTICA Y CULTURAL',
                                        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES, LENGUA CASTELLANA','LICENCIATURA EN EDUCACION BILINGUE','LICENCIATURA EN MUSICA Y DANZA','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: LENGUA CASTELLANA E IDIOMA EXTRANJERO: INGLES',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES,LENGUA CASTELLANA','LICENCIATURA LENGUAS EXTRANJERAS CON ENFASIS EN INGLES','LICENCIATURA EN BILINGUISMO CON ENFASIS EN ESPAÑOL E INGLES','LICENCIATURA EN ESPAÑOL E INGLÉS','LICENCIATURA EN CIENCIAS NATURALES: FÍSICA QUÍMICA Y BIOLOGÍA',
                                        'LICENCIATURA EN EDUCACION BASICA, ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTA','LICENCIATURA EN HUMANIDADES, LENGUA CASTELLANA E INGLES','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN EDUCACIÓN ARTÍSTICA Y CULTURAL: MÚSICA','LICENCIATURA EN ARTES PLASTICAS Y VISUALES',
                                        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN  HUMANIDADES, LENGUA CASTELLANA E INGLES','LICENCIATURA EN  FILOSOFIA','LICENCIATURA EN EDUCACIÓN: ARTES PLÁSTICAS','LICENCIATURA EN EDUCACION PREESCOLAR CON ENFASIS EN INGLES','LICENCIATURA EN FILOSOFIA Y HUMANIDADES',
                                        'LICENCIATURA EN EDUCACIÓN BILINGÜE- ESPAÑOL E INGLES','LICENCIATURA EN MATEMATICAS Y TECNOLOGIAS DE LA INFORMACION','LICENCIATURA EN MATEMÁTICAS Y FÍSICA','LICENCIATURA EN QUIMICA Y EDUCACION AMBIENTAL','LICENCIATURA EN BIOLOGÍA','LICENCIATURA EN MÚSICA','LICENCIATURA EN FILOSOFIA, ETICA Y VALORES HUMANOS',
                                        'LICENCIATURA EN DANZA CLASICA','LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN EDUCACIÓN ARTÍSTICA Y CULTURA','LICENCIATURA EN EDUCACION INICIAL','LICENCIATURA EN EDUCACION RELIGIOSA CON ENFASIS EN BIBLIA','LICENCIATURA EN EDUCACIÓN BÁSICA PRIMARIA CON ÉNFASIS EN INGLÉS Y EN FRANCÉS',
                                        'LICENCIATURA EN EDUCACION ENFASIS EN INFORMATICA','LICENCIATURA EN  LITERATURA Y LENGUA CASTELLANA','LICENCIATURA EN TECNOLOGÍA E INFORMÁTICA','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICAS E INFORMATICA','LICENCIATURA EN MATEM¿TICAS','LICENCIATURA  EN  CIENCIAS  SOCIALES',
                                        'LICENCIATURA EN HUMANIDADES','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION  FISICA, DEPORTES Y RECREACION','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EL AREA DE TECNOLOGIA E INFORMATICA','LICENCIATURA EN FILOSOF¿A Y HUMANIDADES','LICENCIATURA EN PEDAGOGIA REEDUCATIVA',
                                        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES: ESPAÑOL E INGLÉS','LICENCIATURA EN EDUCACION FISICA, DEPORTE Y RECREACION ENFASIS RURAL','LICENCIATURA EN EDUCACION ENFASIS EN CIENCIAS SOCIALES','LICENCIATURA EN EDUCACION BASICA PRIMARIA:ENFASIS EN CIENCIAS SOCIALES',
                                        'LICENCIATURA EN EDUCACON FISICA, RECREACION Y DEPORTES','LICENCIATURA EN CIENCIAS DE LA EDUCACION','LICENCIATURA EN ARTES ESC¿NICAS','LICENCIATURA EN ARTE Y FOLKLORE','LICENCIATURA EN EDUCACION BASICA ENFASIS EN CIENCIAS SOCIALES','LICENCIATURA EN FÍSICA','LICENCIATURA EN ESPAÑOL Y FILOLOGÍA',
                                        'LICENCIATURA EN EDUCACI¿N F¿SICA RECREACI¿N Y DEPORTES'],
                       'COMUNICACION':['COMUNICACION SOCIAL','COMUNICACION SOCIAL- PERIODISMO','COMUNICACION SOCIAL Y PERIODISMO','COMUNICACIÓN SOCIAL','COMUNICACION SOCIAL - PERIODISMO','COMUNICACION SOCIALY PERIODISMO','COMUNICACION SOCIAL-PERIODISMO','COMUNICACION SOCIAL -  PERIODISMO','COMUNICACION AUDIOVISUAL',
                                                'COMUNICACION AUDIOVISUAL Y MULTIMEDIOS','COMUNICACION Y LENGUAJES AUDIOVISUALES','COMUNICACION SOCIAL  - PERIODISMO','COMUNICACION Y RELACIONES CORPORATIVAS','CINE Y COMUNICACIÓN DIGITAL','COMUNICACION GRAFICA PUBLICITARIA','COMUNICACION PUBLICITARIA','DISEÑO DE LA COMUNICACION GRAFICA',
                                                'COMUNICACION','COMUNICACION SOCIAL PERIODISMO','COMUNICACIÓN AUDIOVISUAL','COMUNICACION VISUAL','COMUNICACIONES','COMUNICACION AUDIOVISUAL Y MULTIMEDIAL','COMUNICACION Y PERIODISMO','COMUNICACION ORGANIZACIONAL','COMUNICACIÓN SOCIAL Y MEDIOS DIGITALES','COMUNICACIÓN SOCIAL - PERIODISMO',
                                                'DISEÑO DE COMUNICACIÓN','COMUNICACIÓN Y ENTRETENIMIENTO DIGITAL','COMUNICACIÓN PUBLICITARIA','COMUNICACION SOCIAL     Y     PERIODISMO','PROFESIONAL EN COMUNICACION SOCIAL - PERIODISMO','COMUNICACIÓN CORPORATIVA Y RELACIONES PÚBLICAS','COMUNICACIÓN AUDIOVISUAL Y DIGITAL','COMUNICACIÓN VISUAL Y MULTIMEDIA',
                                                'COMUNICACIÓN CORPORATIVA','COMUNICACIÓN SOCIAL Y PERIODISMO','PERIODISMO Y OPINION PUBLICA','PERIODISMO','CINE Y TELEVISION','PROFESIONAL EN CINE Y TELEVISION','CINE Y TELEVISIÓN','DIRECCION Y PRODUCCION DE CINE Y TELEVISION','PROFESIONAL EN MEDIOS AUDIOVISUALES','DIRECCION Y PRODUCCION DE MEDIOS AUDIOVISUALES',
                                                'CINE Y AUDIOVISUALES'],
                       'CIENCIAS_SALUD':['MEDICINA','MEDICINA VETERINARIA Y ZOOTECNIA','MEDICINA VETERINARIA','ODONTOLOGIA','ODONTOLOGÍA','FISIOTERAPIA','ENFERMERIA','ENFERMERÍA','FARMACIA','QUIMICA Y FARMACIA','OPTOMETRIA','OPTOMETRÍA','NUTRICION Y DIETETICA','NUTRICIÓN Y DIETÉTICA','NUTRICIÓN Y DIETETICA','INSTRUMENTACION QUIRURGICA',
                                               'INSTRUMENTACION QUIRURGICA PROFESIONAL','SALUD OCUPACIONAL','ADMINISTRACION DE LA SEGURIDAD Y SALUD OCUPACIONAL','ADMINISTRACIÓN DE LA SALUD OCUPACIONAL','GERONTOLOGIA','QUIMICA FARMACEUTICA','QUÍMICA FARMACÉUTICA','BACTERIOLOGIA Y LABORATORIO CLINICO','QU¿MICA FARMAC¿UTICA','PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL'],
                       'CIENCIAS_ECONOMICAS':['ECONOMIA','ECONOMÍA','ECONOMIA Y NEGOCIOS INTERNACIONALES','ECONOMIA Y FINANZAS INTERNACIONALES','FINANZAS Y NEGOCIOS INTERNACIONALES','FINANZAS Y COMERCIO INTERNACIONAL','ADMINISTRACION EN FINANZAS Y NEGOCIOS INTERNACIONALES','FINANZAS Y COMERCIO EXTERIOR','NEGOCIOS Y FINANZAS INTERNACIONALES',
                                              'FINANZAS Y RELACIONES INTERNACIONALES','ADMINISTRACION Y FINANZAS','PROFESIONAL EN FINANZAS Y NEGOCIOS INTERNACIONALES','FINANZAS','ADMINISTRACION DE EMPRESAS Y FINANZAS','CONTADURIA PUBLICA Y FINANZAS INTERNACIONALES','COMERCIO INTERNACIONAL Y FINANZAS','PROFESIONAL EN BANCA Y FINANZAS','BANCA Y FINANZAS',
                                              'COMERCIO Y FINANZAS INTERNACIONALES','BANCA Y FINANZAS INTERNACIONALES','CONTADURIA PUBLICA','CONTADURÍA PÚBLICA','CONTADURIA PÚBLICA','PROFESIONAL EN CONTADURIA PUBLICA','CONTADURÍA PUBLICA','CONTADURIA PUBLICA ENFASIS EN SISTEMAS Y ECONOMIA SOLIDARIA','CONTADURIA PUBLICA Y FINANZAS INTERNACIONALES',
                                              'CONTADURIA P¿BLICA','CONTADURÍA PÚBLICA.','CONTADURÍA INTERNACIONAL','CONTADURIA','NEGOCIOS INTERNACIONALES','ADMINISTRACION DE NEGOCIOS INTERNACIONALES','ADMINISTRACION EN FINANZAS Y NEGOCIOS INTERNACIONALES','MARKETING Y NEGOCIOS INTERNACIONALES','MERCADEO Y NEGOCIOS INTERNACIONALES','ADMINISTRACIÓN DE NEGOCIOS INTERNACIONALES',
                                              'NEGOCIOS INTERNACIONALES SEGUNDO CICLO','PROFESIONAL EN MARKETING Y NEGOCIOS INTERNACIONALES','PROFESIONAL EN NEGOCIOS INTERNACIONALES','COMERCIO Y NEGOCIOS INTERNACIONALES','ADMINISTRACION Y NEGOCIOS INTERNACIONALES','3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES','ADMINISTRACIÓN Y NEGOCIOS INTERNACIONALES','ADMINISTRACIÒN DE NEGOCIOS INTERNACIONALES',
                                              'ADMINISTRACION EN NEGOCIOS INTERNACIONALES','MARKETING Y NEGOCIOS INTERNACIONALES','PUBLICIDAD Y MARKETING CREATIVO','MARKETING Y LOGISTICA','PROFESIONAL EN MARKETING INTERNACIONAL','MARKETING','PUBLICIDAD Y MARKETING DIGITAL','ADMINISTRACION PUBLICA TERRITORIAL','ADMINISTRACION PUBLICA','ADMINISTRACIÓN PÚBLICA',
                                              'COMERCIO INTERNACIONAL','COMERCIO EXTERIOR','ADMINISTRACIÓN DE COMERCIO EXTERIOR','ADMINISTRACION DEL COMERCIO INTERNACIONAL','COMERCIO Y FINANZAS INTERNACIONALES','ADMINISTRACION DE COMERCIO EXTERIOR','ADMINISTRACION DE EMPRESAS','ADMINISTRACIÓN DE EMPRESAS','PROFESIONAL EN ADMINISTRACIÓN DE EMPRESAS','ADMINISTRACION DE EMPRESAS AGROPECUARIAS',
                                              'ADMINISTRACION DE EMPRESAS COMERCIALES','ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS','ADMINISTRACIÓN DE EMPRESAS TURÍSTICAS Y HOTELERAS','ADMINISTRACIÓN DE EMPRESAS TURÍSTICAS','ADMINISTRACION DE EMPRESAS TURISTICAS','ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS','ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL',
                                              'ADMINISTRACIÓN DE EMPRESAS AGROPECUARIAS','ADMINISTRACIÓN DE EMPRESAS TURISTICAS','ADMINISTRACIÓN DE EMPRESAS - CICLO PROFESIONAL','ADMINISTRACION DE EMPRESAS TURISTICA','ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES','ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES','ADMINISTRACIÓN DE EMPRESAS AGROINDUSTRIALES','ADMINISTRACIÓN DE EMPRESAS TURISTICAS POR CICLOS - CICLO PROFESIONAL',
                                              'ADMINISTRACIÓN LOGÍSTICA','ADMINISTRACION EN MERCADEO Y LOGISTICA INTERNACIONALES','ADMINISTRACION LOGISTICA','ADMINISTRACION EN LOGISTICA Y PRODUCCION','LOGISTICA Y MERCADEO ORGANIZACIONAL','ADMINISTRACIÓN LOGISTICA','LOGISTICA EMPRESARIAL','ADMINISTRACION DE MERCADEO Y LOGISTICA INTERNACIONALES','MERCADEO','MERCADEO Y PUBLICIDAD','ADMINISTRACION DE MERCADEO',
                                              'ADMINISTRACION COMERCIAL Y DE MERCADEO','PUBLICIDAD Y MERCADEO','PROFESIONAL EN MERCADEO Y PUBLICIDAD','ADMINISTRACION EN MERCADEO Y LOGISTICA INTERNACIONALES','MERCADEO INTERNACIONAL Y PUBLICIDAD','PROFESIONAL EN MERCADEO','MERCADEO NACIONAL E INTERNACIONAL','MERCADEO PUBLICIDAD Y VENTAS','PROFESIONAL EN MERCADEO EMPRESARIAL','PROFESIONAL EN PUBLICIDAD Y MERCADEO',
                                              'MERCADEO Y VENTAS','ADMINISTRACION DE MERCADEO Y LOGISTICA INTERNACIONALES','MERCADEO AGROINDUSTRIAL','ECONOM¿A','HOTELERIA Y TURISMO ECOLOGICO','HOTELERIA Y TURISMO','GESTIÓN EN ECOLOGÍA Y TURISMO','NEGOCIOS Y RELACIONES INTERNACIONALES','GESTION EMPRESARIAL'],
                       'ADMINISTRACION':['ADMINISTRACIÓN EN SALUD OCUPACIONAL','ADMINISTRACIÓN FINANCIERA','ADMINISTRACION FINANCIERA','ADMINISTRACION DE NEGOCIOS','ADMINISTRACION PUBLICA','ADMINISTRACION EN SALUD','ADMINISTRACION TURISTICA Y HOTELERA','ADMINISTRACION Y DIRECCION DE EMPRESAS','ADMINISTRACION DE SERVICIOS DE SALUD','ADMINISTRACIÓN INTEGRAL DE RIESGOS DE SEGURIDAD Y SALUD EN EL TRABAJO',
                                         'ADMINISTRACIÓN EN SEGURIDAD Y SALUD EN EL TRABAJO','ADMINISTRACION POLICIAL','ADMINISTRACION AMBIENTAL','ADMINISTRACIÓN TECNÓLOGICA','ADMINISTRACION INDUSTRIAL','ADMINISTRACION HUMANA','ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES','ADMINISTRACIÓN DE LA SEGURIDAD Y SALUD   EN EL TRABAJO','ADMINISTRACION DEL MEDIO AMBIENTE','ADMINISTRACION EN SALUD: ENFASIS EN GESTION DE SERVICIOS DE SALUD Y  ENFASIS EN GESTION SANITARIA Y AMBIENTAL',
                                         'ADMINISTRACION MARITIMA Y FLUVIAL','ADMINISTRACIÓN COMERCIAL Y FINANCIERA','ADMINISTRACION DE SISTEMAS INFORMATICOS','ADMINISTRACIÓN COMERCIAL','ADMINISTRACION BANCARIA Y FINANCIERA','ADMINISTRACION DEPORTIVA','ADMINISTRACION DE LA SEGURIDAD SOCIAL','ADMINISTRACIÓN PÚBLICA','ADMINISTRACION','ADMINISTRACION AGROPECUARIA','ADMINISTRACION DEL TURISMO SOSTENIBLE','ADMINISTRACION AERONAUTICA',
                                         'ADMINISTRACION  FINANCIERA','ADMINISTRACION FINANCIERA Y DE SISTEMAS','ADMINISTRACIÓN TURÍSTICA','ADMINISTRACION COMERCIAL','ADMINISTRACION MARITIMA','ADMINISTRACION DE INSTITUCIONES DE SERVICIO','ADMINISTRACIÓN TURÍSTICA Y HOTELERA','ADMINISTRACION Y GESTION AMBIENTAL','ADMINISTRACIÓN HOTELERA Y DE SERVICIOS','ADMINISTRACION DE SISTEMAS INFORMÁTICOS','ADMINISTRACION MARITIMA Y PORTUARIA',
                                         'ADMINISTRACION & SERVICIO','ADMINISTRACIÓN DE AGRONEGOCIOS','ADMINISTRACION INFORMÁTICA','ADMINISTRACIÓN MARITIMA Y PORTUARIA','ADMINISTRACIÓN AGROINDUSTRIAL','ADMINISTRACIÓN DE OBRAS DE ARQUITECTURA','ADMINISTRACION DE GESTISN HUMANA','ADMINISTRACION DE HOTELERIA Y TURISMO','ADMINISTRACION TURÍSTICA','ADMINISTRACION DE SISTEMAS','ADMINISTRACIÓN EN SERVICIOS DE SALUD','ADMINISTRACION DE LA CALIDAD',
                                         'ADMINISTRACION INFORMATICA','ADMINISTRACION TECNOLOGICA','GESTION EN ECOLOGIA Y TURISMO'],
                       'CIENCIAS_EXACTAS':['MATEMATICAS','MATEMATICAS CON ENFASIS EN ESTADISTICA','MATEMÁTICAS','MATEMÁTICAS APLICADAS Y CIENCIAS DE LA COMPUTACIÓN','MATEMATICAS APLICADAS','FISICA','FÍSICA','QUIMICA','QUIMICA.','QUÍMICA','ASTRONOMIA','ESTADISTICA',
                                           'ESTADISTICA','ESTADÍSTICA','CIENCIAS DE LA COMPUTACIÓN E INTELIGENCIA ARTIFICIAL','CIENCIAS AMBIENTALES','CIENCIAS DE LA COMPUTACION'],
                       'CIENCIAS_NATURALES':['GEOCIENCIAS','GEOLOG¿A','QUIMICA AMBIENTAL','BIOLOGIA','MICROBIOLOGIA','BIOLOGÍA','MICROBIOLOGIA Y BIOANALISIS','MICROBIOLOGIA INDUSTRIAL','BIOLOGIA MARINA','MICROBIOLOGIA INDUSTRIAL Y AMBIENTAL','BIOLOGIA APLICADA','MICROBIOLOGÍA','BIOLOGIA AMBIENTAL','MICROBIOLOGÍA INDUSTRIAL Y AMBIENTAL',
                                             'BIOLOGÍA AMBIENTAL','MICROBIOLOGÍA Y BIOANÁLISIS','ZOOTECNIA','ECOLOGIA','ECOLOGIA DE ZONAS COSTERAS','ECOLOGÍA','BIOQUIMICA','GEOGRAFIA','GEOGRAFIA DEL DESARROLLO REGIONAL Y AMBIENTAL','GEOGRAFIA APLICADA A LA ORGANIZACION DEL ESPACIO Y PLANIFICACION REGIONAL','OCEANOGRAFIA FISICA','OCEANOGRAFÍA','PROFESIONAL EN INVESTIGACIÓN CRIMINAL',
                                             'INVESTIGACION CRIMINAL','INVESTIGACIÓN CRIMINAL','ARQUEOLOGIA','AGRONOMIA','AGRONOMIA DEL TROPICO HUMEDO'],
                       'CIENCIAS_HUMANAS':['FILOSOFIA','FILOSOFIA Y LETRAS','RECREACION','FILOSOFÍA','PROFESIONAL EN FILOSOFIA','FILOSOFÍA Y HUMANIDADES','FILOSOFIA Y HUMANIDADES','ESTUDIOS EN FILOSOFIA','ESTUDIOS LITERARIOS','LITERATURA','CREACION LITERARIA',
                                           'LINGUISTICA Y LITERATURA','ESTUDIOS LITERARIOS Y EDICION','FILOLOGIA E IDIOMAS','ESPAÑOL Y FILOLOGIA CLASICA','FILOLOGIA HISPANICA','LETRAS - FILOLOGIA HISPANICA','TRADUCCION INGLES-FRANCES-ESPAÑOL','LENGUAS MODERNAS','LENGUAS MODERNAS','PROFESIONAL EN LENGUAS MODERNAS','PROGRAMA PROFESIONAL UNIVERSITARIO EN LENGUAS EXTRANJERAS',
                                           'LENGUAS Y CULTURA','LENGUAS MODERNAS Y CULTURA','HISTORIA','HISTORIA Y ARCHIVISTICA','HISTORIA DEL ARTE','HISTORIA Y PATRIMONIO','ESTUDIOS Y GESTIÓN CULTURAL','ESTUDIOS Y GESTION CULTURAL','ANTROPOLOGIA','ANTROPOLOGÍA','RELACIONES INTERNACIONALES','RELACIONES INTERNACIONALES Y ESTUDIOS POLITICOS','GOBIERNO Y RELACIONES INTERNACIONALES',
                                           'POLITICA Y RELACIONES INTERNACIONALES','CIENCIA POLITICA Y RELACIONES INTERNACIONALES','CIENCIA POLITICA, GOBIERNO Y RELACIONES INTERNACIONALES','TERAPIA OCUPACIONAL','PSICOLOGIA','PSICOLOGÍA','PROFESIONAL EN PSICOLOGÍA','TRABAJO SOCIAL','PROFESIONAL EN TRABAJO SOCIAL','DERECHO','DERECHO Y CIENCIAS POLITICAS','DERECHO Y CIENCIAS SOCIALES','DERECHO Y CIENCIAS POLÍTICAS',
                                           'LENGUAJES Y ESTUDIOS SOCIOCULTURALES','GESTION Y DESARROLLO URBANOS','GESTION CULTURAL Y COMUNICATIVA','ESTUDIOS Y GESTION CULTURAL','ESTUDIOS Y GESTIÓN CULTURAL','GESTIÓN CULTURAL','GESTIÓN  DE SISTEMAS DE INFORMACIÓN','TURISMO','ARCHIVISTICA','SISTEMAS DE INFORMACIÓN, BIBLIOTECOLOGÍA Y ARCHIVÍSTICA','CIENCIA DE LA INFORMACION Y BIBLIOTECOLOGIA','CIENCIA DE LA INFORMACION - BIBLIOTECOLOGIA',
                                           'BIBLIOTECOLOGIA','TEOLOGIA','TEOLOGÍA','CIENCIA POLITICA Y GOBIERNO','CIENCIA POLÍTICA','CIENCIA POLÍTICA Y GOBIERNO','CIENCIAS DE LA INFORMACION Y LA DOCUMENTACION','CIENCIAS BIBLICAS','ESTUDIOS DE ARTES LIBERALES EN CIENCIAS SOCIALES','PEDAGOGÍA','JURISPRUDENCIA' ],
                       'ARTES_DISEÑO':['ARTES PLASTICAS','ARTES VISUALES','ARTES MUSICALES','ARTES ESCENICAS','ARTES ESCÉNICAS','ARTES AUDIOVISUALES','ARTES PLASTICAS Y VISUALES','ARTES PLÁSTICAS Y VISUALES','ARTES DE LA ESCENA','ARTES DE LA GRABACION Y PRODUCCION MUSICAL','ARTES PLÁSTICAS','ARTES PLASTICAS CON ENFASIS EN PROCESOS BIDIMENSIONALES Y TRIDIMENSIONALES','BELLAS ARTES',
                                       'ARTE DRAMATICO','ARTE','ARTE DANZARIO','DANZA','DANZA Y DIRECCIÓN COREOGRÁFICA','TEATRO MUSICAL','MUSICA','ESTUDIOS MUSICALES','FORMACION MUSICAL','MÚSICA','MUSICA INSTRUMENTAL','INTERPRETACION MUSICAL','MAESTRO EN MÚSICA','MAESTRO EN MUSICA','MUSICA INSTRUMENTO','MUSICA- CANTO','DISEÑO GRAFICO','DISEÑO GRÁFICO','DISEÑO DE MODAS','DISEÑO','DISEÑO VISUAL',
                                       'DISEÑO DE VESTUARIO','DISEÑO INTERIOR','DISEÑO DE COMUNICACION VISUAL','DISEÑO DIGITAL Y MULTIMEDIA','DISEÑO DE LA COMUNICACION GRAFICA','DISEÑO Y PRODUCCION DE MODA','DISEÑO Y GESTIÓN DE LA MODA','DISEÑO DE MEDIOS  INTERACTIVOS','DISEÑO INTERACTIVO','DISEÑO DE ESPACIOS - ESCENARIO','PROFESIONAL EN DISEÑO GRAFICO','DISEÑO DE ESPACIOS','DISEÑO VISUAL DIGITAL',
                                       'DISEÑO DE COMUNICACIÓN','DISEÑO DE ESPACIOS Y ESCENARIOS','PROFESIONAL  EN DISEÑO GRAFICO','PROFESIONAL EN GESTION DEL DISEÑO','PROFESIONAL UNIVERSITARIO DISEÑO GRÁFICO DIGITAL','DISE¿O Y ADMINISTRACI¿N DE NEGOCIOS DE LA MODA','PROFESIONAL EN DISEÑO INDUSTRIAL','PROFESIONAL EN DISEÑO VISUAL','DISEÑO DE MODA','DISE¿¿O DE COMUNICACI¿¿N GR¿¿FICA',
                                       'PROFESIONAL EN DISEÑO GRÁFICO','DISEÑO GRÀFICO','PROFESIONAL EN DISEÑO DE INTERIORES','DISEÑO Y GESTIÓN DE ESPACIOS','PROFESIONAL EN DISEÑO DE ALTA COSTURA','DISE¿O CROSSMEDIA','DISEÑO CROSSMEDIA','DISEÑO Y GESTIÓN DE LA MODA Y EL TEXTIL','PROFESIONAL EN DISEÑO DE ESPACIOS','DISEÑO Y REALIZACIÓN DE MEDIOS DIGITALES','DISENO GRAFICO','PROFESIONAL EN DISEÑO DE MODAS',
                                       'PROFESIONAL EN DISEÑO DE VESTUARIO','DISEÑO Y GESTIÓN DEL PRODUCTO','PROFESIONAL UNIVERSITARIO EN DISEÑO TEXTIL PARA LA INDUSTRIA','DISEÑO GRAFICO Y MULTIMEDIAL','DISEÑO DE MODAS Y ALTA COSTURA','GASTRONOMIA','PROFESIONAL EN GASTRONOMIA',
                                       'PROFESIONAL  EN GASTRONOMÍA Y  ALTA COCINA','CULINARIA Y GASTRONOMÍA','ARTES CULINARIAS Y GASTRONOMÍA','GASTRONOMÍA','GASTRONOMIA Y COCINA PROFESIONAL','PROFESIONAL EN GASTRONOMÍA','PROFESIONAL EN GASTRONOMÍA Y CULINARIA','PROFESIONAL EN GASTRONOM¿A','PROFESIONAL EN ARTES GASTRONOMICAS'],
                       'ARQUITECTURA':['ARQUITECTURA','CONSTRUCCION Y GESTION EN ARQUITECTURA','CONSTRUCCION EN ARQUITECTURA E INGENIERIA','CONSTRUCCIONES CIVILES','CONSTRUCCION'],
                       'CIENCIA_DEPORTES':['PROFESIONAL EN ENTRENAMIENTO DEPORTIVO','CULTURA FISICA, DEPORTE Y RECREACION','DEPORTE','PROFESIONAL EN DEPORTE','CIENCIAS DEL DEPORTE Y DE LA EDUCACION FISICA','ENTRENAMIENTO DEPORTIVO','CIENCIAS DEL DEPORTE Y  LA RECREACION',
                                           'ACTIVIDAD FISICA Y DEPORTE','CIENCIAS DEL DEPORTE','PROFESIONAL EN ACTIVIDAD FISICA Y DEPORTE','CULTURA FISICA Y DEPORTE','ADMINISTRACION DEPORTIVA','DEPORTE Y ACTIVIDAD FISICA','CIENCIAS DEL DEPORTE Y LA ACTIVIDAD FISICA','PROFESIONAL EN ACTIVIDAD FÍSICA Y DEPORTE',
                                           'DEPORTE Y CULTURA FISICA','GESTI¿N DEPORTIVA','PROFESIONAL EN CIENCIAS DEL DEPORTE Y LA EDUCACION FISICA','DEPORTE Y ACTIVIDADA FISICA','FORMACION PROFESIONAL EN DEPORTE']}

# con el diccionario definido no se puede dar un mapeo normal, entonces se realiza con una funcion y los valores que no se incluyeron de forma manual en los grupos se le da un grupo aparte para esos faltantes
def mapear_programa(programa):
    for categoria, programas in ESTU_PRGM_ACADEMICO.items():
        if programa in programas:
            return categoria
    return 'NO_DEFINIDO'


In [10]:
# organizo y confirmo que todos mis datos categoricos sean de tipo str para poder trabajar la columna 'ESTU_PRGM_DEPARTAMENTO'
df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].astype(str)

# hacer un diccionario con la columna 'ESTU_PRGM_ACADEMICO' donde se pueda generar unas categorias y disminuir la cantidad de valores unicos
ESTU_PRGM_DEPARTAMENTO = {'ANDINA':['ANTIOQUIA','BOYACA','CALDAS','CUNDINAMARCA','HUILA','NORTE SANTANDER','QUINDIO','RISARALDA',
                                    'SANTANDER','TOLIMA','BOGOTÁ','CAUCA'],
                          'CARIBE':['ATLANTICO','BOLIVAR','CESAR','CORDOBA','LA GUAJIRA','MAGDALENA','SAN ANDRES','SUCRE'],
                          'PACIFICO':['CHOCO','VALLE','NARIÑO'],
                          'ORINOQUIA': ['ARAUCA','CASANARE','META'],
                          'AMAZONICA': ['AMAZONAS','CAQUETA','GUAVIARE','VAUPES','PUTUMAYO']}

# con el diccionario definido no se puede dar un mapeo normal, entonces se realiza con una funcion y los valores que no se incluyeron de forma manual en los grupos se le da un grupo aparte para esos faltantes
def mapear_departamento(programa):
    for categoria, programas in ESTU_PRGM_DEPARTAMENTO.items():
        if programa in programas:
            return categoria
    return 'NO_DEFINIDO'

In [11]:
def clean_df(df):
 # cambiar la columna ID como un indice o un identificador unico
 df.set_index("ID")

 # separar la columna 'periodo' en año y periodo
 df['AÑO'] = df['PERIODO'].astype(str).str[:4]
 df['PERIODO'] = df['PERIODO'].astype(str).str[-1]

 # columnas 'año' y 'periodo' definidos
 df['AÑO'] = df['AÑO'].astype(int)
 df['PERIODO'] = df['PERIODO'].astype(int)
 df[['AÑO', 'PERIODO']].head()

 # columna 'ESTU_VALORMATRICULAUNIVERSIDAD'
 ESTU_VALORMATRICULAUNIVERSIDAD = {'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Menos de 500 mil': 0.25,
    'Entre 500 mil y menos de 1 millón': 0.75,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Más de 7 millones': 7.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'No pagó matrícula': 0,}
 df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(ESTU_VALORMATRICULAUNIVERSIDAD)
 media_mapeo1 = df['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()
 df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(media_mapeo1)

 #  columna 'ESTU_HORASSEMANATRABAJA'
 ESTU_HORASSEMANATRABAJA = {'0':0,
          'Menos de 10 horas':5,
          'Entre 11 y 20 horas':15.5,
          'Entre 21 y 30 horas':25.5,
          'Más de 30 horas':45}
 df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map(ESTU_HORASSEMANATRABAJA)
 media_mapeo2 = df['ESTU_HORASSEMANATRABAJA'].mean()
 df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna(media_mapeo2)

 # columna 'FAMI_TIENEINTERNET'
 df.fillna({'FAMI_TIENEINTERNET': 'Si'}, inplace=True)
 FAMI_TIENEINTERNET_ = {'Si':1,'No':0}
 df['FAMI_TIENEINTERNET']=df['FAMI_TIENEINTERNET'].map(FAMI_TIENEINTERNET_)

 # columna 'ESTU_PAGOMATRICULAPROPIO'
 df.fillna({'ESTU_PAGOMATRICULAPROPIO': 'No'}, inplace=True)
 ESTU_PAGOMATRICULAPROPIO = {'Si':1,'No':0}
 df['ESTU_PAGOMATRICULAPROPIO']=df['ESTU_PAGOMATRICULAPROPIO'].map(ESTU_PAGOMATRICULAPROPIO)

 # columna 'FAMI_EDUCACIONMADRE'
 df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].astype(str)
 df.FAMI_EDUCACIONMADRE = ['No info' if i in ['No sabe', 'No Aplica', 'Ninguno','nan'] else i for i in df.FAMI_EDUCACIONMADRE.values]
 df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna('No info')
 FAMI_EDUCACIONMADRE = {'No info':0,
          'Primaria incompleta':1,
          'Primaria completa':2,
          'Secundaria (Bachillerato) incompleta':3,
          'Secundaria (Bachillerato) completa':4,
          'Técnica o tecnológica incompleta':5,
          'Técnica o tecnológica completa':6,
          'Educación profesional incompleta':7,
          'Educación profesional completa':8,
          'Postgrado':9}
 df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].map(FAMI_EDUCACIONMADRE)

 # columna 'FAMI_EDUCACIONPADRE'
 df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].astype(str)
 df.FAMI_EDUCACIONPADRE = ['No info' if i in ['No sabe', 'No Aplica', 'Ninguno','nan'] else i for i in df.FAMI_EDUCACIONPADRE.values]
 df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna('No info')
 FAMI_EDUCACIONPADRE = {'No info':0,
          'Primaria incompleta':1,
          'Primaria completa':2,
          'Secundaria (Bachillerato) incompleta':3,
          'Secundaria (Bachillerato) completa':4,
          'Técnica o tecnológica incompleta':5,
          'Técnica o tecnológica completa':6,
          'Educación profesional incompleta':7,
          'Educación profesional completa':8,
          'Postgrado':9}
 df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].map(FAMI_EDUCACIONPADRE)
 df['FAMI_EDUCACIONPADRE'].head()

 # columna 'ESTU_PRGM_ACADEMICO'
 df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(mapear_programa)
 categorias_expand = pd.get_dummies(df['ESTU_PRGM_ACADEMICO'], prefix='ACADEMICO')
 categorias_expand = categorias_expand.astype(int)
 df = pd.concat([df, categorias_expand], axis=1)
 df = df.drop(columns=['ESTU_PRGM_ACADEMICO'])

 # columna 'ESTU_PRGM_DEPARTAMENTO'
 df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].apply(mapear_departamento)
 # ahora separaremos las columnas mediante one-hot encoding
 categorias_expand = pd.get_dummies(df['ESTU_PRGM_DEPARTAMENTO'], prefix='DEPARTAMENTO')
 categorias_expand = categorias_expand.astype(int)
 df = pd.concat([df, categorias_expand], axis=1)
 df = df.drop(columns=['ESTU_PRGM_DEPARTAMENTO'])
 return df

# Transformar los datos de train y test juntos

In [12]:
dtr = pd.read_csv("train.csv")
dts = pd.read_csv("test.csv")
lentr = len(dtr)
dtr.shape, dts.shape

((692500, 12), (296786, 12))

In [13]:
# hacer un diccionario para la columna 'FAMI_ESTRATOVIVIENDA_dtr	' eliminando la cadena de texto y dando valores enteros
FAMI_ESTRATOVIVIENDA_dtr = {'Sin Estrato':0,
                         'Estrato 1':1, 'Estrato 2':2,
                         'Estrato 3':3, 'Estrato 4':4,
                         'Estrato 5':5, 'Estrato 6':6}
dtr['FAMI_ESTRATOVIVIENDA']=dtr['FAMI_ESTRATOVIVIENDA'].map(FAMI_ESTRATOVIVIENDA_dtr)

# para poder darle un valor a los valores nulos en este caso, use condiciones, donde tuve en cuenta otra columna que pueda estar
# relacionada con el estrato de alguna forma, como lo puede ser el valor de la matricula del estudiante
estrato1 = {0, 0.25, 0.75}
estrato5 = {4.75, 6.25}
def asignar_estrato(row):
    if pd.isna(row['FAMI_ESTRATOVIVIENDA']):
        if row['ESTU_VALORMATRICULAUNIVERSIDAD'] in estrato1:
            return 1
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == 1.75:
            return 2
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == 3.25:
            return 3
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].mean():
            return 4
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] in estrato5:
            return 5
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == 7.75:
            return 6
    return row['FAMI_ESTRATOVIVIENDA']

    dtr['FAMI_ESTRATOVIVIENDA'] = dtr.apply(asignar_estrato, axis=1)

# si queda algun valor nulo que no responda a la función se le dara un valor de forma manual
dtr.fillna({'FAMI_ESTRATOVIVIENDA': 2}, inplace=True)

In [14]:
# hacer un diccionario para la columna 'FAMI_ESTRATOVIVIENDA_dts' eliminando la cadena de texto y dando valores enteros
FAMI_ESTRATOVIVIENDA_dts = {'Sin Estrato':0,
                         'Estrato 1':1, 'Estrato 2':2,
                         'Estrato 3':3, 'Estrato 4':4,
                         'Estrato 5':5, 'Estrato 6':6}
dts['FAMI_ESTRATOVIVIENDA']=dts['FAMI_ESTRATOVIVIENDA'].map(FAMI_ESTRATOVIVIENDA_dts)

# para poder darle un valor a los valores nulos en este caso, use condiciones, donde tuve en cuenta otra columna que pueda estar
# relacionada con el estrato de alguna forma, como lo puede ser el valor de la matricula del estudiante
estrato1 = {0, 0.25, 0.75}
estrato5 = {4.75, 6.25}
def asignar_estrato(row):
    if pd.isna(row['FAMI_ESTRATOVIVIENDA']):
        if row['ESTU_VALORMATRICULAUNIVERSIDAD'] in estrato1:
            return 1
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == 1.75:
            return 2
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == 3.25:
            return 3
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == dts['ESTU_VALORMATRICULAUNIVERSIDAD'].mean():
            return 4
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] in estrato5:
            return 5
        elif row['ESTU_VALORMATRICULAUNIVERSIDAD'] == 7.75:
            return 6
    return row['FAMI_ESTRATOVIVIENDA']

    dts['FAMI_ESTRATOVIVIENDA'] = dts.apply(asignar_estrato, axis=1)

# si queda algun valor nulo que no responda a la función se le dara un valor de forma manual
dts.fillna({'FAMI_ESTRATOVIVIENDA': 2}, inplace=True)

In [15]:
#  hacer un diccionario para la columna 'RENDIMIENTO_GLOBAL_'
RENDIMIENTO_GLOBAL_ = {'medio-bajo':0,'bajo':1,
                       'medio-alto':2,'alto':3}
dtr['RENDIMIENTO_GLOBAL']=dtr['RENDIMIENTO_GLOBAL'].map(RENDIMIENTO_GLOBAL_)
dtr['RENDIMIENTO_GLOBAL'].head()

,RENDIMIENTO_GLOBAL
0,2
1,1
2,1
3,3
4,0


In [16]:
# se elimina la columna 'unnamed' de test ya que no aporta nada
dts = dts.drop('Unnamed: 0', axis=1)

In [17]:
# se preparan los datos para el entrenamiento
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((dtr[source_cols], dts[source_cols]))
all_data.index = range(len(all_data))
all_data = clean_df(all_data)

Xtr, ytr = all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xts      = all_data.iloc[lentr:].values

print (Xtr.shape, ytr.shape)
print (Xts.shape)


(692500, 28) (692500,)
(296786, 28)


In [18]:
all_data.head()

,ID,PERIODO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,AÑO,...,ACADEMICO_CIENCIA_DEPORTES,ACADEMICO_COMUNICACION,ACADEMICO_INGENIERIAS,ACADEMICO_LICENCIATURAS,ACADEMICO_NO_DEFINIDO,DEPARTAMENTO_AMAZONICA,DEPARTAMENTO_ANDINA,DEPARTAMENTO_CARIBE,DEPARTAMENTO_ORINOQUIA,DEPARTAMENTO_PACIFICO
0,904256,2,6.25,5.0,3.0,1,5,9,0,2021,...,0,0,0,0,0,0,1,0,0,0
1,645256,2,3.25,0.0,3.0,0,6,5,0,2021,...,0,0,0,0,0,0,0,1,0,0
2,308367,3,3.25,45.0,3.0,1,4,4,0,2020,...,0,0,0,0,0,0,1,0,0,0
3,470353,5,4.75,0.0,4.0,1,0,4,0,2019,...,0,0,0,0,0,0,1,0,0,0
4,989032,2,3.25,25.5,3.0,1,2,2,0,2021,...,0,0,0,0,0,0,1,0,0,0


# Modelo solucion

In [46]:
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
import xgboost as xgb

In [47]:
# modelos viables para trabajar en la solucion
lgb_ = lgb.LGBMClassifier(objective='multiclass', num_class=4, metric='multi_logloss')
xgb_ = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [48]:
# validacion del modelo mas optimo
print (cross_val_score(lgb_, Xtr, ytr))
print (cross_val_score(xgb_, Xtr, ytr))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062285 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 28
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057716 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 28
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start 

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:57:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:58:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:59:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:59:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:00:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

[0.41575451 0.41158123 0.41507581 0.41459206 0.4112491 ]


# Entrenar los datos completos

In [49]:
# se escogio el modelo lightgbm
lgb_.fit(Xtr, ytr)
preds_ts = lgb_.predict(Xts)
preds_ts

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.131201 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Number of data points in the train set: 692500, number of used features: 28
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.387092
[LightGBM] [Info] Start training from score -1.395031
[LightGBM] [Info] Start training from score -1.371991


array([1, 2, 3, ..., 2, 3, 3])

In [50]:
submission = pd.DataFrame([dts.ID, preds_ts], index=['ID', 'RENDIMIENTO_GLOBAL']).T
submission

,ID,RENDIMIENTO_GLOBAL
0,550236,1
1,98545,2
2,499179,3
3,782980,1
4,785185,1
...,...,...
296781,496981,0
296782,209415,3
296783,239074,2
296784,963852,3


In [51]:
#  darle valores categoricos a la columna 'RENDIMIENTO_GLOBAL_'
mapeo_inverso = {0:'medio-bajo',1:'bajo',
                       2:'medio-alto',3:'alto'}
submission['RENDIMIENTO_GLOBAL']=submission['RENDIMIENTO_GLOBAL'].map(mapeo_inverso)
submission['RENDIMIENTO_GLOBAL'].head(10)

,RENDIMIENTO_GLOBAL
0,bajo
1,medio-alto
2,alto
3,bajo
4,bajo
5,bajo
6,alto
7,alto
8,medio-bajo
9,medio-bajo


In [52]:
submission.to_csv("submission_predict.csv", index=False)

In [53]:
!head submission_predict.csv

ID,RENDIMIENTO_GLOBAL
550236,bajo
98545,medio-alto
499179,alto
782980,bajo
785185,bajo
58495,bajo
705444,alto
557548,alto
519909,medio-bajo
